In [3]:
import os
from pathlib import Path

from PIL import Image
import torch
from facenet_pytorch import MTCNN
from tqdm import tqdm

In [4]:


# ====== 1. SET PATH SESUAI PUNYAMU ======
INPUT_ROOT  = Path("./data")
OUTPUT_ROOT = Path("./data_face224")

OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

# ====== 2. INISIALISASI DETECTOR WAJAH ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mtcnn = MTCNN(
    keep_all=False,   # kita cuma ambil 1 wajah per gambar
    device=device
)

VALID_EXT = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".heic"}

# ====== 3. LOOP SEMUA KELAS (NAMA MAHASISWA) ======
class_dirs = [d for d in INPUT_ROOT.iterdir() if d.is_dir()]

for class_dir in tqdm(class_dirs, desc="Kelas"):
    class_name = class_dir.name

    out_class_dir = OUTPUT_ROOT / class_name
    out_class_dir.mkdir(parents=True, exist_ok=True)

    img_files = [
        f for f in class_dir.iterdir()
        if f.is_file() and f.suffix.lower() in VALID_EXT
    ]

    for img_path in img_files:
        try:
            img = Image.open(img_path).convert("RGB")
            w_img, h_img = img.size

            # 1) DETEKSI WAJAH (bounding box)
            boxes, probs = mtcnn.detect(img)

            if boxes is not None:
                # ambil wajah pertama
                x1, y1, x2, y2 = boxes[0]

                # tambahkan margin & bikin kotak (square)
                cx = (x1 + x2) / 2.0
                cy = (y1 + y2) / 2.0
                side = max(x2 - x1, y2 - y1) * 1.2  # 20% margin

                x1_new = int(max(cx - side / 2, 0))
                y1_new = int(max(cy - side / 2, 0))
                x2_new = int(min(cx + side / 2, w_img))
                y2_new = int(min(cy + side / 2, h_img))

                # 2) CROP DARI GAMBAR ASLI
                face = img.crop((x1_new, y1_new, x2_new, y2_new))
            else:
                # kalau tidak terdeteksi, pakai gambar asli
                face = img

            # 3) RESIZE JADI 224x224
            face = face.resize((224, 224), Image.BILINEAR)

            # 4) SIMPAN SEBAGAI IMAGE BIASA
            out_path = out_class_dir / img_path.name
            face.save(out_path)

        except Exception as e:
            print(f"[WARNING] Gagal proses {img_path}: {e}")


Kelas:   0%|          | 0/4 [00:00<?, ?it/s]

Kelas: 100%|██████████| 4/4 [00:07<00:00,  1.99s/it]
